## 3.BAG OF WORDS


  We convert text to a numerical representation called a feature vector. 
  A feature vector can be as simple as a list of numbers.
  
  The bag-of-words model is one of the feature extraction algorithms for text.

1.The first step in this model is defining the vocabulary

2.The second step is to convert sentences into a frequency vector based on the vocabulary.

In [11]:
#Reading data from document
import pandas as pd
df_pre=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")
df=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")

In [12]:
from sklearn import linear_model

In [13]:
#Adjustments to be done for the data
df['content']=df['content'].str.replace(",", "")

In [14]:
df['content']=df['content'].str.replace("'","")

In [15]:
#creating a list for all content
l=[]
for i in range(0,10000):
    l.append(df['content'].iloc[i])
l

['[lijiang tourism sz002033 stock stock financ invest bank recommend baoy share divid half earn princip group]',
 '[chen ling ding red envelop make money abil grab red envelop fight technolog i grab red envelop cash issu chen ding ling kan hongyan burst happi valentin "s" day fan togeth around red envelop â•® â¯ 3â¯ â•\\xadhttp tcnrzdivjf]',
 '[taobao sucker industri ga fire send children nima blind]',
 '[aspect say know everyth laugh cri]',
 '[over zhang]',
 '[lifetim my favorit main bodi hook three togeth burst happi black tong googl]',
 '[on uniqlo dress room sound insul good bad share know almost]',
 '[so ordinari everyday scene also much longer accumul one day got look pictur i feel burst nostalgia age rememb pain accompani]',
 '[overh red xxiii tanggua children stick grab red envelop win i grab littl _ big red overh wang xiansen issu year goat yet come good luck come red envelop tri luck http tcnrzggwg6]',
 '[there good thing share flash note note todo best chine softwar go downl

In [16]:
#code for bag of words model
import numpy as np
import re

#for building vocabulary
def tokenize_sentences(sentences):
    words = []
    for sentence in sentences:
        w = extract_words(sentence)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

def extract_words(sentence):
    ignore_words = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    words = re.sub("[^\w]", " ",  sentence).split() #nltk.word_tokenize(sentence)
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned    

#function which returns feature vector
def bagofwords(sentence, words):
    sentence_words = extract_words(sentence)
    # frequency word count
    bag = np.zeros(len(words),dtype=int)
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag)

In [17]:
#building the vocabulary for the list created
vocabulary1 = tokenize_sentences(l)

In [18]:
l1 = [x for x in vocabulary1 if not (x.isdigit() or x[1:].isdigit())]

In [19]:
b=pd.DataFrame()

In [20]:
#constructing bag of words
a=[]

for i in range(0,10000):
    #b.append(bagofwords(df['content'].iloc[i], vocabulary1),ignore_index=True)
    a.append(bagofwords(df['content'].iloc[i], vocabulary1))

In [21]:
type(a)

list

In [22]:
bow=np.asarray(a)

In [23]:
df_pol=pd.read_csv("E:\\DMA_PRE\\weibo_polarity.csv")

In [24]:
type(bow)
print(bow.shape)

(10000, 16792)


In [25]:
df_pol.columns

Index(['Unnamed: 0', 'u_id', 'm_id', 'forward_count', 'comment_count',
       'like_count', 'content', 'date', 'time', 'content_media_count',
       'content_#_count', 'content_@_count', 'content_?_count',
       'content_!_count', 'content_length', 'content_emoji_count', 'hour',
       'min', 'sec', 'forward_min', 'forward_max', 'forward_median',
       'forward_mean', 'comment_min', 'comment_max', 'comment_median',
       'comment_mean', 'like_min', 'like_max', 'like_median', 'like_mean',
       'Unnamed: 0.1', 'content_spchar', 'non_emoji_content', 'en_content',
       'Unnamed: 1', 'url_rem', 'contentwurl', 'polarity'],
      dtype='object')

In [26]:

bow1=np.insert(bow,16791,df_pol["content_media_count"],axis=1)
bow2=np.insert(bow1,16792,df_pol["forward_median"],axis=1)
bow3=np.insert(bow2,16793,df_pol["comment_median"],axis=1)
bow4=np.insert(bow3,16794,df_pol["like_median"],axis=1)
bow5=np.insert(bow4,16795,df_pol["polarity"],axis=1)



In [29]:
train_bow=bow5[0:8000]
pred_bow=bow5[8001:10000]

# Linear Regression Model using BOW and additional factors

In [31]:
X_train1=train_bow
X_test1=pred_bow
Y_train1=train_df[["forward_count","like_count","comment_count"]]
Y_test1=predict_df[["forward_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [32]:
tbow=bow5[0:8000]

In [33]:
cvbow=bow5[8001:10000]

In [34]:
type(tbow)

numpy.ndarray

In [35]:
df_pre.shape
dftrain=df_pre[0:8000]
dfcv=df_pre[8001:10000]

In [36]:
Y_test1=dfcv[["forward_count","like_count","comment_count"]]

In [37]:
np.savetxt("E://DMA_PRE//weibo_predict_resultbow.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow.csv")

In [38]:
train_real_pred = Y_test1
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score on the training set:9.13%


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [39]:
result

,forward_count,comment_count,like_count
0,5.0,0.0,0.0
1,157.0,0.0,34.0
2,0.0,0.0,0.0
3,0.0,0.0,-0.0
4,0.0,0.0,-0.0
5,0.0,0.0,0.0
6,12.0,432.0,184.0
7,68.0,43.0,27.0
8,71.0,40.0,37.0
9,19.0,7.0,10.0


In [40]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import statsmodels.api as sm

In [41]:
#from evaluation import precision

In [42]:
df1=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")
df_bow=pd.read_csv("E:\\DMA_PRE\\bow_df.csv")

In [43]:
#result = pd.concat([df1, df], axis=1)

In [44]:
import import_ipynb
from evaluation import precision
from runTime import runTime

# BAG OF WORDS USING COUNTER VECTORIZER


In [45]:
df1=pd.read_csv('E:\\DMA_PRE\\pre_bow.csv')
train_df=df1[0:8000]
train_df.shape

(8000, 10)

In [46]:
predict_df=df1[8001:10000]

In [47]:
train_l=[]
for i in range(0,8000):
    train_l.append(df_pre['content'].iloc[i])
len(train_l)

8000

In [48]:
pred_l=[]
for i in range(8001,10000):
    pred_l.append(df_pre['content'].iloc[i])
len(pred_l)

1999

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
vect=CountVectorizer()

In [51]:
vect.fit(train_l)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [52]:
x=vect.transform(train_l)

In [53]:
#vocabulary=vect.get_feature_names()
#print(vocabulary)

In [54]:
print(x[0,:])

  (0, 1593)	1
  (0, 1608)	1
  (0, 3688)	1
  (0, 3923)	1
  (0, 4535)	1
  (0, 5181)	1
  (0, 5286)	1
  (0, 6171)	1
  (0, 6991)	1
  (0, 9310)	1
  (0, 9783)	1
  (0, 10657)	1
  (0, 11334)	2
  (0, 11648)	1
  (0, 13110)	1


In [55]:
type(x)

scipy.sparse.csr.csr_matrix

In [56]:
arrbow=x.toarray()

In [57]:
vect = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 100)

In [58]:
print(vect)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [59]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import precision_score

# Model 1 

In [60]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()
off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_forward = predict_df.forward_count

X_train1=off_train_data_features
X_test1= off_test_data_features
Y_train1=dftrain[["forward_count","like_count","comment_count"]]
Y_test1=dfcv[["forward_count","like_count","comment_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))
np.savetxt("E://DMA_PRE//weibo_predict_resultbow1.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow1.csv")
result1=result1.abs()
result1=result1.astype(int)
train_real_pred = Y_test1
train_real_pred['fp']=result1['forward_count'].values
train_real_pred['cp']=result1['comment_count'].values
train_real_pred['lp']=result1['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))


Score on the training set:15.43%


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

# Model 2

In [61]:
print(x.shape)

(8000, 14753)


In [62]:
x[1]

<1x14753 sparse matrix of type '<class 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [63]:
x[2]

<1x14753 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [64]:
train=df_pol[0:8000]
cv=df_pol[8001:10000]

In [65]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()

off_train_data_features1=np.insert(off_train_data_features,100,train["content_media_count"],axis=1)
off_train_data_features2=np.insert(off_train_data_features1,101,train["forward_median"],axis=1)
off_train_data_features3=np.insert(off_train_data_features2,102,train["comment_median"],axis=1)
off_train_data_features4=np.insert(off_train_data_features3,103,train["like_median"],axis=1)
#off_train_data_features5=np.insert(off_train_data_features4,100,train["polarity"],axis=1)
off_train_data_features6=np.insert(off_train_data_features4,104,train["content_emoji_count"],axis=1)
#off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_features1=np.insert(off_test_data_features,100,cv["content_media_count"],axis=1)
off_test_data_features2=np.insert(off_test_data_features1,101,cv["forward_median"],axis=1)
off_test_data_features3=np.insert(off_test_data_features2,102,cv["comment_median"],axis=1)
off_test_data_features4=np.insert(off_test_data_features3,103,cv["like_median"],axis=1)
#off_test_data_features5=np.insert(off_test_data_features4,100,cv["polarity"],axis=1)
off_test_data_features6=np.insert(off_test_data_features4,104,cv["content_emoji_count"],axis=1)
#off_test_data_forward = predict_df.forward_count

X_train1=off_train_data_features6
X_test1= off_test_data_features6
Y_train1=dftrain[["forward_count","like_count","comment_count"]]
Y_test1=dfcv[["forward_count","like_count","comment_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow3.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result3=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow3.csv")
result3=result3.abs()
result3=result3.astype(int)
train_real_pred = Y_test1
train_real_pred['fp']=result3['forward_count'].values
train_real_pred['cp']=result3['comment_count'].values
train_real_pred['lp']=result3['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))


Score on the training set:15.89%


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-



# Model 3

In [92]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()

off_train_data_features1=np.insert(off_train_data_features,100,train["content_media_count"],axis=1)
off_train_data_features2=np.insert(off_train_data_features1,101,train["forward_median"],axis=1)
off_train_data_features3=np.insert(off_train_data_features2,102,train["comment_median"],axis=1)
off_train_data_features4=np.insert(off_train_data_features3,103,train["like_median"],axis=1)
off_train_data_features5=np.insert(off_train_data_features4,104,train["polarity"],axis=1)

#off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_features1=np.insert(off_test_data_features,100,cv["content_media_count"],axis=1)
off_test_data_features2=np.insert(off_test_data_features1,101,cv["forward_median"],axis=1)
off_test_data_features3=np.insert(off_test_data_features2,102,cv["comment_median"],axis=1)
off_test_data_features4=np.insert(off_test_data_features3,103,cv["like_median"],axis=1)
off_test_data_features5=np.insert(off_test_data_features4,104,cv["polarity"],axis=1)

#off_test_data_forward = predict_df.forward_count

X_train1=off_train_data_features4
X_test1= off_test_data_features4
Y_train1=dftrain["forward_count"]
Y_test1=dfcv["forward_count"]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow4.csv",pred1,delimiter=',',header="forward_count",comments="")
result4=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow4.csv")
result4=result4.abs()
result4=result4.astype(int)

train_real_pred = Y_test1



In [93]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()

off_train_data_features1=np.insert(off_train_data_features,100,train["content_media_count"],axis=1)
off_train_data_features2=np.insert(off_train_data_features1,101,train["forward_median"],axis=1)
off_train_data_features3=np.insert(off_train_data_features2,102,train["comment_median"],axis=1)
off_train_data_features4=np.insert(off_train_data_features3,103,train["like_median"],axis=1)
off_train_data_features5=np.insert(off_train_data_features4,100,train["polarity"],axis=1)
#off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_features1=np.insert(off_test_data_features,100,cv["content_media_count"],axis=1)
off_test_data_features2=np.insert(off_test_data_features1,101,cv["forward_median"],axis=1)
off_test_data_features3=np.insert(off_test_data_features2,102,cv["comment_median"],axis=1)
off_test_data_features4=np.insert(off_test_data_features3,103,cv["like_median"],axis=1)
off_test_data_features5=np.insert(off_test_data_features4,100,cv["polarity"],axis=1)
#off_test_data_forward = predict_df.forward_count

X_train2=off_train_data_features4
X_test2= off_test_data_features4
Y_train2=dftrain["like_count"]
Y_test2=dfcv["like_count"]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow5.csv",pred1,delimiter=',',header="comment_count",comments="")
result5=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow5.csv")
result5=result5.abs()
result5=result5.astype(int)



In [94]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()

off_train_data_features1=np.insert(off_train_data_features,100,train["content_media_count"],axis=1)
off_train_data_features2=np.insert(off_train_data_features1,101,train["forward_median"],axis=1)
off_train_data_features3=np.insert(off_train_data_features2,102,train["comment_median"],axis=1)
off_train_data_features4=np.insert(off_train_data_features3,103,train["like_median"],axis=1)
off_train_data_features5=np.insert(off_train_data_features4,100,train["polarity"],axis=1)
#off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_features1=np.insert(off_test_data_features,100,cv["content_media_count"],axis=1)
off_test_data_features2=np.insert(off_test_data_features1,101,cv["forward_median"],axis=1)
off_test_data_features3=np.insert(off_test_data_features2,102,cv["comment_median"],axis=1)
off_test_data_features4=np.insert(off_test_data_features3,103,cv["like_median"],axis=1)
off_test_data_features5=np.insert(off_test_data_features4,100,cv["polarity"],axis=1)
#off_test_data_forward = predict_df.forward_count

X_train3=off_train_data_features4
X_test3= off_test_data_features4
Y_train3=dftrain["comment_count"]
Y_test3=dfcv["comment_count"]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow6.csv",pred1,delimiter=',',header="like_count",comments="")
result6=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow6.csv")
result6=result6.abs()
result6=result6.astype(int)



In [95]:
train_real_pred = pd.concat([Y_test1,Y_test2,Y_test3],axis=1)
train_real_pred['fp']=result4['forward_count'].values
train_real_pred['cp']=result5['comment_count'].values
train_real_pred['lp']=result6['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score on the training set:16.20%


In [96]:
train_real_pred

,forward_count,like_count,comment_count,fp,cp,lp
8001,0,0,0,0,0,0
8002,3,0,0,0,0,0
8003,0,1,0,12,12,12
8004,0,0,0,4,4,4
8005,0,0,0,4,4,4
8006,0,0,0,117,117,117
8007,2,0,0,0,0,0
8008,0,0,0,3,3,3
8009,3,1,2,0,0,0
8010,0,0,0,0,0,0


In [147]:
from sklearn.ensemble import GradientBoostingRegressor

In [148]:
train=df_pol[0:8000]
cv=df_pol[8001:10000]

In [149]:
X_train6=train[["content_media_count","forward_median","comment_median","like_median"]]
X_test6= cv[["content_media_count","forward_median","comment_median","like_median"]]
Y_train6=train[["forward_count"]]
Y_test6=cv[["forward_count"]]


In [150]:
gbrt=GradientBoostingRegressor(n_estimators=100) 
gbrt.fit(X_train6, Y_train6) 
y_pred1=gbrt.predict(X_test6)
y_pred1=y_pred1.round()
y_pred1=(np.maximum(y_pred1,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow6.csv",y_pred1,delimiter=',',header="forward_count",comments="")
result6=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow6.csv")
result6=result6.abs()
result6=result6.astype(int)


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [151]:
X_train7=train[["content_media_count","forward_median","comment_median","like_median"]]
X_test7= cv[["content_media_count","forward_median","comment_median","like_median"]]
Y_train7=train[["like_count"]]
Y_test7=cv[["like_count"]]

gbrt=GradientBoostingRegressor(n_estimators=100) 
gbrt.fit(X_train7, Y_train7) 
y_pred2=gbrt.predict(X_test7)
y_pred2=y_pred2.round()
y_pred2=(np.maximum(y_pred2,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow7.csv",y_pred2,delimiter=',',header="like_count",comments="")
result7=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow7.csv")
result7=result7.abs()
result7=result7.astype(int)


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [152]:
X_train8=train[["content_media_count","forward_median","comment_median","like_median"]]
X_test8= cv[["content_media_count","forward_median","comment_median","like_median"]]
Y_train8=train[["comment_count"]]
Y_test8=cv[["comment_count"]]

gbrt=GradientBoostingRegressor(n_estimators=100) 
gbrt.fit(X_train8, Y_train8) 
y_pred3=gbrt.predict(X_test8)
y_pred3=y_pred3.round()
y_pred3=(np.maximum(y_pred3,0.))

np.savetxt("E://DMA_PRE//weibo_predict_resultbow8.csv",y_pred3,delimiter=',',header="comment_count",comments="")
result8=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow8.csv")
result8=result8.abs()
result8=result8.astype(int)


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [157]:
train_real_pred = pd.concat([Y_test6,Y_test8,Y_test7],axis=1)
train_real_pred['fp']=result6['forward_count'].values
train_real_pred['cp']=result8['comment_count'].values
train_real_pred['lp']=result7['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score on the training set:99.51%


In [156]:
train_real_pred 

,forward_count,comment_count,like_count,fp,cp,lp
8001,0,0,0,0,0,0
8002,0,0,0,0,0,0
8003,0,0,0,0,0,0
8004,0,0,0,0,0,0
8005,0,0,0,0,0,0
8006,0,0,0,0,0,0
8007,0,0,0,0,0,0
8008,0,0,0,0,0,0
8009,0,0,0,0,0,0
8010,0,1,0,0,0,0
